In [1]:
%run downoald_and_cleaning_data.ipynb

Downloading...
From (original): https://drive.google.com/uc?id=1UR_MC8Q5K4JSRnSUX5MQ-sIGPjLLnm9f
From (redirected): https://drive.google.com/uc?id=1UR_MC8Q5K4JSRnSUX5MQ-sIGPjLLnm9f&confirm=t&uuid=030d23ac-8f2b-47c7-a08f-14f4ea083cd0
To: /home/onyxia/work/PSSD/Data/data.csv
100%|██████████| 363M/363M [00:04<00:00, 78.3MB/s] 


In [2]:
df = pd.read_csv("data_clean.csv")

In [3]:
df.dropna(subset=['texte'], inplace=True)

In [4]:
df['texte'] = df['texte'].str.lower()

In [5]:
import re
from tqdm import tqdm

tqdm.pandas()
df['texte_clean'] = df['texte'].str.lower().progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['titre_clean'] = df['titre'].str.lower().progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))


100%|██████████| 11970/11970 [00:00<00:00, 258280.31it/s]


In [6]:
import os
os.chdir('..')
df_themes = pd.read_csv('Annotations/theme.csv')

In [7]:
df_merged = df.merge(df_themes, on="identifiant", how="left", suffixes=("", "_manual"))
df_merged["texte_total"] = df_merged["titre"].fillna("") + " " + df_merged["texte"].fillna("")

In [12]:
from transformers import pipeline
from datasets import Dataset

classifier = pipeline(
    "zero-shot-classification",
    model="mtheo/camembert-base-xnli",
    device=0,
    hypothesis_template="Cet article parle de {}."
)
candidate_labels = [
    "actualité",
    "culture",
    "politique",
    "tribune",
    "analyse",
    "société"
]
def batch_classify(df, text_column='texte_total', batch_size=96):
    texts = df[text_column].fillna(" ").tolist()
    texts = [t if isinstance(t, str) and t.strip() != "" else " " for t in texts]
    dataset = Dataset.from_dict({"text": texts})

    def batching(batch):
        results = classifier(
            batch["text"],
            candidate_labels=candidate_labels,
            batch_size=batch_size
        )
        batch_themes = [res["labels"][0] for res in results]
        batch_scores = [res["scores"][0] for res in results]
        return {"theme_pred": batch_themes, "theme_score": batch_scores}

    dataset = dataset.map(
        batching,
        batched=True,
        batch_size=batch_size
    )
    return dataset.to_pandas()


Device set to use cuda:0


In [13]:
def filter_empty_lines(df, text_column='texte_clean'):
    filtered_df = df[~(df[text_column].isna() | (df[text_column].str.strip() == ''))].copy()
    filtered_df = filtered_df.reset_index()  
    print(f"Lignes initiales : {len(df)}")
    print(f"Lignes après filtrage : {len(filtered_df)}")
    return filtered_df

df_filtered = filter_empty_lines(df, text_column='texte_clean')


Lignes initiales : 11970
Lignes après filtrage : 11970


In [14]:
#results = batch_classify(df_filtered, text_column='texte_clean', batch_size=164)

In [ ]:
# Filtrer les articles avec un thème connu pour évaluation
df_to_classify = df_merged[df_merged["theme"].notna()].reset_index(drop=True)
df_predictions = batch_classify(df_to_classify)

# Fusionner les prédictions avec les données d'origine
df_result = pd.concat([df_to_classify, df_predictions[["theme_pred", "theme_score"]]], axis=1)

Map:  74%|███████▍  | 288/390 [01:27<00:31,  3.27 examples/s]

In [ ]:
df_filtered['theme'] = results['theme'].values

In [ ]:
df_filtered.tail(10)

In [ ]:
df_faits = df_filtered[df_filtered['theme']=='faits divers']

In [ ]:
df_faits.shape

In [ ]:
import pandas as pd
df_faits=pd.read_csv('data_faits_divers.csv')

In [ ]:
import matplotlib.pyplot as plt
articles_par_annee = df_faits['annee'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
articles_par_annee.plot(kind='bar')
plt.title("Répartition des articles par année")
plt.xlabel("Année")
plt.ylabel("Nombre d'articles")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
articles_par_annee = df_faits['annee'].value_counts(normalize=True).sort_index() * 100
plt.figure(figsize=(10, 6))
articles_par_annee.plot(kind='bar')
plt.title("Répartition des articles par année (%)")
plt.xlabel("Année")
plt.ylabel("Pourcentage d'articles")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()
plt.savefig('plot.png')

In [ ]:
df_faits.to_csv('data_faits_divers.csv')

In [ ]:
df_filtered.to_csv('data_all.csv')